In this notebook, we will compute spectral differentiation for different  
ensembles of neurons, with different state and window lengths.  
We will then generate some of the plots in the manuscript.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
import os
from os import path
import itertools
import pickle

import numpy as np
import pandas as pd
import scipy as sp
from scipy import signal as ssn
from tqdm.auto import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from glob import glob

from ipympl.backend_nbagg import Canvas
Canvas.header_visible.default_value = False

from differentiation import spectral_differentiation as specD
from differentiation import spectral_states

In [3]:
data_directory = (
    '/allen/programs/braintv/workgroups/tiny-blue-dot/'
    'differentiation/refactor/data'
)

hierarchy = {
    'Input' : -100,
    'stimulus' : -100,
    'Stim' : -100,
    'TH' : -10,
    'LG' : -9,
    'LGv' : -8,
    'LGd' : -7,
    'LP' : -6,
    'THx' : -5,
    'THx_VISp' : -4,
    'VISp' : 0,
    'VISpl' : 2,
    'VISl' : 4,
    'VISli' : 6,
    'VISrl' : 8,
    'VISal' : 10,
    'VISpm' : 12,
    'VISam' : 14,
    'VISpor' : 16,
    'VISa' : 18,
    'SC' : 24,
    'VISmma' : 20,
    'VISmmp' : 20,
    'VIS' : 22,
    'HVAs' : 21,
    'VisCtx' : 21.5,
    'AllVis' : 22,
    'PF' : 25,
    'MB' : 30,
    'hipp' : 38,
    'CAx' : 39,
    'CA' : 40,
    'CA1' : 41,
    'CA2' : 42,
    'CA3' : 43,
    'DG' : 50,
}

hierarchy_score = {
    'LGd':-0.5150279628298357,
    'VISp':-0.35733209934482374,
    'VISl':-0.09388855125761343,#LM
    'VISrl':-0.05987132463908328,
    'LP':0.10524780962600731,
    'VISal':0.15221797920142832,
    'VISpm':0.32766807486511995,
    'VISam':0.440986074378801
}

colors = {
    x : i/len(hierarchy) for i, x in enumerate(hierarchy.keys())
}

layer_depths = {
    'L1' : 100,
    'L2/3' : 210,
    'L4' : 120,
    'L5' : 220,
    'L6' : 200,
}

region_sets = {
    'VisCtx' : ['VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam'],
    'HVAs' : ['VISl', 'VISrl', 'VISal', 'VISpm', 'VISam'],
    'AllVis' : [
        'LGd', 'LP', 'TH', 'VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam'
    ],
    'THx' : ['LGd', 'LP', 'TH'],
    'hipp' : ['CA', 'CA1', 'CA2', 'CA3', 'DG', 'DG-mo', 'DG-po', 'DG-sg'],
}

relevant_regions = [
    'LGd', 'LP', 'TH', 'VISp', 'VISl', 'VISrl', 'VISal', 'VISpm', 'VISam'
]

stim_colors_bg = {
    'spontaneous' : cm.Greys(0.3, 0.3),
    'gabors' : cm.Reds(0.7, 0.3),
    'flashes' : cm.Reds(0.3, 0.3),
    'drifting_gratings' : cm.Blues(0.8, 0.3),
    'drifting_gratings_contrast' : cm.Blues(0.99, 0.3),
    'static_gratings' : cm.Blues(0.5, 0.3),
    'natural_movie_three' : cm.Greens(0.9, 0.3),
    'natural_movie_one' : cm.Greens(0.6, 0.3),
    'natural_movie_one_shuffled' : cm.Purples(0.6, 0.1),
    'Spontaneous' : cm.Greys(0.6, 0.3),
    'Artificial (simple)' : cm.Reds(0.6, 0.3),
    'Artificial (complex)' : cm.Blues(0.8, 0.3),
    'Natural' : cm.Greens(0.8, 0.3),
}

stim_colors = {
    'spontaneous' : cm.Greys(0.3, 0.8),
    'gabors' : cm.Reds(0.3, 0.8),
    'flashes' : cm.Reds(0.8, 0.8),
    'drifting_gratings' : cm.Blues(0.5, 0.8),
    'drifting_gratings_contrast' : cm.Blues(0.7, 0.8),
    'static_gratings' : cm.Blues(0.9, 0.8),
    'natural_movie_three' : cm.Greens(0.75, 0.8),
    'natural_movie_two' : cm.Greens(0.6, 0.8),
    'natural_movie_one' : cm.Greens(0.45, 0.8),
    'natural_movie_one_shuffled' : cm.Purples(0.3, 0.8),
    'Spontaneous' : cm.Greys(0.6, 0.8),
    'Artificial (simple)' : cm.Reds(0.6, 0.8),
    'Artificial (complex)' : cm.Blues(0.8, 0.8),
    'Natural' : cm.Greens(0.8, 0.8),
}

stim_cat_colors = {
    'spontaneous' : cm.Greys(0.8, 0.8),
    'simple' : cm.Reds(0.8, 0.8),
    'complex' : cm.Blues(0.8, 0.8),
    'natural' : cm.Greens(0.8, 0.8),
    'shuffled' : cm.Purples(0.4, 0.8)
    
}

stim_cat_colors_bg = {
    'spontaneous' : cm.Greys(0.5, 0.5),
    'simple' : cm.Reds(0.5, 0.5),
    'complex' : cm.Blues(0.5, 0.5),
    'natural' : cm.Greens(0.5, 0.5),
    'shuffled' : cm.Purples(0.4, 0.5)
    
}

stimulus_categories = {
    'drifting_gratings' : 'complex',
    'drifting_gratings_contrast' : 'simple',
    'flashes' : 'simple',
    'gabors' : 'simple',
    'natural_movie_one_shuffled' : 'shuffled',
    'natural_movies' : 'natural',
    'natural_movie_one' : 'natural',
    'natural_movie_three' : 'natural',
    'spontaneous' : 'spontaneous',
    'static_gratings' : 'complex'
}

stim_by_putative_meaning = {
    'stimulus_name' : [
        'spontaneous', 'natural_movie_one_shuffled', 'flashes', 'gabors',
        'drifting_gratings_contrast', 'drifting_gratings', 'static_gratings',
        'natural_movie_one', 'natural_movie_three', 'natural_movies'
    ],
    'stimulus_category' : [
        'spontaneous', 'simple', 'shuffled', 'complex', 'natural'
    ]
}

In [4]:
session_ids = [
    path.basename(x)
    .strip('.pkl')
    .strip('fr_') for x in glob(
        path.join(data_directory, 'fr_*')
    )
]

In [5]:
# ensembles of neurons to apply differentiation to
rs = True
unit_set = []
unit_set  = [
    f'region == "%s" & snr > 2.5 & RS == {rs}'%reg for reg in [
        'VISp', 'VISl', 'VISal', 'VISam', 'VISpm', 'VISrl', 'LGd', 'LP'
    ]
]
unit_set += [
    f'region in @region_sets.get("%s") & snr > 2.5 & RS == {rs}'%s for s in region_sets.keys()
]
unit_set += [
    f'layer == "%s" & region == "%s" & snr > 2.5 & RS == {rs}'%(l, r) for l, r in itertools.product(
        ['L2/3', 'L4', 'L5', 'L6'], ['VISp', 'VISl', 'VISal', 'VISpm', 'VISam', 'VISrl']
    )
]
unit_set += [
    f'layer == "%s" & region in @region_sets.get("%s") & snr > 2.5 & RS == {rs}'%(l, r) for l, r in itertools.product(
        ['L2/3', 'L4', 'L5', 'L6'], list(region_sets.keys())[:2]
    )
]
# --------------------------------------------------------------------------------------------------------------------
# unit_set += [
#     'layer == "%s" & snr > 2.5 & RS == True'%l for l in [
#         'L1', 'L2/3', 'L4', 'L5', 'L6'
#     ]
# ]
# unit_set += [
#     'layer in @ecephys.layer_sets.get("%s") & snr > 2.5 & RS == True'%s for s in ecephys.layer_sets.keys()
# ]
# unit_set += [
#     'layer in @ecephys.layer_sets.get("%s") & region == "%s" & snr > 2.5 & RS'%(s, r) for s, r in itertools.product(
#         ecephys.layer_sets.keys(), ['VISp', 'VISl', 'VISal', 'VISpm', 'VISam', 'VISrl']
#     )
# ]
# --------------------------------------------------------------------------------------------------------------------
print(
    f'Differentiation will be computed for {len(unit_set)} ensembles per session.'
)

Differentiation will be computed for 45 ensembles per session.


In [6]:
# set up parameters for computing spectral differentiatioon
state_length_scaling_exponent = 2
sampling_rate = 200
state_lengths = [0.1, 0.15, 0.3]#[0.005, 0.01, 0.015, 0.03, 0.06, 0.1, 0.15, 0.3, 0.5, 1, 3, 10]
win_lengths = [3]#, 9, 30]
param_sets = [
    {
        'STATE_LENGTH' : s,
        'WINDOW' : w,
        'RESOLUTION' : w, # not used, defaults to w in this notebook
    } for w, s in itertools.product(win_lengths, state_lengths) if w/s<=600
]
print(f'Computing for {len(param_sets)} parameter sets per session per ensemble.')

Computing for 3 parameter sets per session per ensemble.


In [7]:
def mfrD(unit_fr_local, sampling_rate, state_length, window_length):
    unit_state_mfr = np.reshape(
        unit_fr_local.T,
        (
            unit_fr_local.shape[1], -1,
            int(state_length*sampling_rate)
        )
    ).mean(axis=2)#.transpose(0, 2, 1)
    # display(unit_state_mfr.shape)
    state_mfr = unit_state_mfr.mean(axis=0)
    # display(state_mfr.shape)
    window_state_mfr = np.reshape(state_mfr, (-1, int(window_length/state_length)))
    # window_state_mfr.shape
    diffn_var = window_state_mfr.std(axis=1)
    return diffn_var

def boxD(unit_fr, sampling_rate, state_length, window_length):
    _spec = ssn.spectrogram(
        unit_fr, fs=sampling_rate, window=('boxcar'),
        nperseg=int(state_length*sampling_rate), noverlap=0,
        detrend=False, mode='psd', scaling='spectrum', axis=0
    )[-1]
    # reshape the result to combine all neurons into a single vector x time
    _spec_reshaped = np.reshape(_spec, (_spec.shape[0]*_spec.shape[1], -1))
    # drop some time at the end to make the shape compatible with an integral number of windows
    _spec_reshaped = _spec_reshaped[:, :-int(_spec_reshaped.shape[1]%int(window_length / state_length))]
    # reshape to divide the array into vectors x states x windows
    _spec_final = np.reshape(_spec_reshaped, (_spec_reshaped.shape[0], -1, int(window_length / state_length)))
    # compute pairwise distances between states for all windows
    pdists = np.array([sp.spatial.distance.pdist(_spec_final[:, i, :].T) for i in range(_spec_final.shape[1])])
    # compute median distance for all windows and normalize by number of units
    diffn = np.median(pdists, axis=-1) / np.sqrt(unit_fr.shape[1])
    return diffn

def tukD(unit_fr, sampling_rate, state_length, window_length):
    _spec = ssn.spectrogram(
        unit_fr, fs=sampling_rate, window=('tukey', 1),
        nperseg=int(state_length*sampling_rate), noverlap=0,
        detrend=False, mode='psd', scaling='spectrum', axis=0
    )[-1]
    # reshape the result to combine all neurons into a single vector x time
    _spec_reshaped = np.reshape(_spec, (_spec.shape[0]*_spec.shape[1], -1))
    # drop some time at the end to make the shape compatible with an integral number of windows
    _spec_reshaped = _spec_reshaped[:, :-int(_spec_reshaped.shape[1]%int(window_length / state_length))]
    # reshape to divide the array into vectors x states x windows
    _spec_final = np.reshape(_spec_reshaped, (_spec_reshaped.shape[0], -1, int(window_length / state_length)))
    # compute pairwise distances between states for all windows
    pdists = np.array([sp.spatial.distance.pdist(_spec_final[:, i, :].T) for i in range(_spec_final.shape[1])])
    # compute median distance for all windows and normalize by number of units
    diffn = np.median(pdists, axis=-1) / np.sqrt(unit_fr.shape[1])
    return diffn

In [8]:
def load_fr(session):
    return pd.read_pickle(
        path.join(data_directory, f'fr_{session}.pkl')
    )

def load_units(session):
    return pd.read_pickle(
        path.join(data_directory, f'units_{session}.pkl')
    )

def load_stimulus_table(session):
    return pd.read_pickle(
        path.join(data_directory, f'stimulus_{session}.pkl')
    )

def load_running(session):
    return pd.read_pickle(
        path.join(data_directory, f'running_{session}.pkl')
    )

units_bar = None
param_bar = None
def compute_differentiation(session, param_sets, unit_set, min_units=10, diff_fn='specD'):
    global units_bar, param_bar
    if units_bar is None:
        units_bar = tqdm()
    if param_bar is None:
        param_bar = tqdm()
    units_bar.reset(len(unit_set))
    units_bar.set_description(session)
    units_bar.refresh()
    
    differentiation = []

    # load firing rates
    firing_rates = load_fr(session)

    # load units
    up = load_units(session)
    up['idx'] = range(len(up))

    # compute differentiation for each ensemble and parameter set
    for i, units_name in enumerate(unit_set):
        units = up[up.eval(units_name)]['idx'].values
#         print(units_name, ':', len(units))
        if len(units)<min_units:
            units_bar.update()
            continue
        
        # extract firing rate for selected units
        unit_fr = firing_rates[units]

        # normalize firing rates wrt mean enseble firing rate over entire session
        unit_fr = unit_fr / unit_fr.values.mean()
#         display(unit_fr.shape)

        
        param_bar.reset(len(param_sets))
        param_bar.set_description('params')
        param_bar.refresh()
        for params in param_sets:
#             print(params)
            window_length = params['WINDOW']
            state_length = params['STATE_LENGTH']

            if diff_fn=='boxD':
                try:
                    df = boxD(unit_fr, sampling_rate, state_length, window_length)
                except:
                    continue
            elif diff_fn=='tukD':
                try:
                    df = tukD(unit_fr, sampling_rate, state_length, window_length)
                except:
                    continue
            else:
                # reshape for single shot spectral differentiation calculation
                unit_fr_local = unit_fr.values[:int(
                    unit_fr.shape[0]/(int(window_length*sampling_rate))
                )*int(window_length*sampling_rate)]
    #             display(unit_fr_local.shape)
            if diff_fn=='specD':
                unit_fr_local = np.reshape(
                    unit_fr_local.T,
                    (
                        unit_fr_local.shape[1], -1,
                        int(window_length*sampling_rate)
                    )
                ).transpose(1, 0, 2)
    #             display(unit_fr_local.shape)

                # compute spectral differentiation
                try:
                    df = specD(
                        unit_fr_local,
                        sample_rate=sampling_rate,
                        window_length=state_length
                    )
    #                 display(df.shape)
                except:
                    df = np.ones((unit_fr_local.shape[0], 2))*np.nan
    #             display(df.shape)

                # get median differentiation
                df = np.median(df, axis=1)
            elif diff_fn=='mfrD':
                df = mfrD(
                    unit_fr_local,
                    sampling_rate=sampling_rate,
                    state_length=state_length,
                    window_length=window_length
                )

#             return df

            try:
                # put it into a nice series indexed by time
                times = unit_fr.index
                times = np.linspace(
                    times[0],
                    int(times[-1]/window_length)*int(window_length),
                    df.shape[0], False, dtype=int
                ) + window_length/2
    #             times = times + np.diff(times).mean()/2
        #             display(times)
                df = pd.Series(
                    df, index=times,
                    name=(window_length, state_length,
                          f'{units_name} & n_units = {len(units)}')
                )
    #             display(df)
                differentiation.append(df)
            except:
                continue
            param_bar.update()
        param_bar.refresh()
        param_bar.reset()
        units_bar.update()
    if len(differentiation)>0:
        differentiation = pd.concat(differentiation, axis=1)
        differentiation = differentiation.sort_index(axis=1)
        differentiation = differentiation.rename_axis(
            columns=['window_length', 'state_length', 'region']
        )
    else:
        differentiation = None
    return differentiation

---
differentiation with tapers

In [9]:
firing_rates = load_fr(session_ids[0])
up = load_units(session_ids[0])
up['idx'] = range(len(up))
units = up[up.eval(unit_set[0])]['idx'].values
unit_fr = firing_rates[units]
unit_fr = unit_fr / unit_fr.values.mean()
unit_fr

,710,715,719,720,727,735,736,737,742,756,...,1075,1076,1081,1083,1089,1124,1126,1127,1130,1139
0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.005000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.015000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.020000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10227.643695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10227.648695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10227.653695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10227.658695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
unit_fr.max().max()

53.5240663580184

In [11]:
# get spectral states and see if they are properly bounded
# highest firing units: 1081, 1075, 735
# unit_fr.mean().sort_values()
data = unit_fr[[1081]].values[10000:19000].T[np.newaxis]
# data = np.ones(600)[np.newaxis, np.newaxis]
ss = spectral_states(200, 0.3, data)[0] / 200**2 / 0.3**2
ss.shape

max signal: 38.80494810956334
max FFT component: 613.653420794681
max FFT component squared: 376570.5208530138


(150, 31)

In [107]:
ss[ss>30]

array([ 37.61327118,  36.68894711,  50.42220522,  37.55858086,
        53.76991318,  70.39037841,  44.9421319 ,  37.12249073,
        54.88769256,  38.88213033,  51.56885169,  49.10074469,
       104.60292246])

In [16]:
_ss = sp.signal.spectrogram(data, fs=200, window='box', nperseg=60, noverlap=0, scaling='spectrum')[2][0][0]

In [13]:
__ss = sp.signal.stft(data, fs=200, window='box', nperseg=60, noverlap=0, boundary=None)[2][0][0]
print(np.abs(__ss).max())
__ss = np.abs(__ss)**2

10.227557013244683


In [17]:
_ss.shape

(31, 150)

In [99]:
__ss.shape

(31, 150)

In [18]:
_ss / __ss

/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[1.05356483e-32, 7.31642244e-33, 1.32657118e-32, ...,
        4.82654755e-33, 7.43088885e-33, 4.85481033e-33],
       [2.00000000e+00, 2.00000000e+00, 2.00000000e+00, ...,
        2.00000000e+00, 2.00000000e+00, 2.00000000e+00],
       [2.00000000e+00, 2.00000000e+00, 2.00000000e+00, ...,
        2.00000000e+00, 2.00000000e+00, 2.00000000e+00],
       ...,
       [2.00000000e+00, 2.00000000e+00, 2.00000000e+00, ...,
        2.00000000e+00, 2.00000000e+00, 2.00000000e+00],
       [2.00000000e+00, 2.00000000e+00, 2.00000000e+00, ...,
        2.00000000e+00, 2.00000000e+00, 2.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00]])

In [100]:
ss

array([[7.48754023e-01, 7.33337644e-01, 6.88947586e-01, ...,
        5.54320245e-05, 7.28473699e-05, 7.95784911e-05],
       [7.48754023e-01, 7.33337644e-01, 6.88947586e-01, ...,
        5.54320245e-05, 7.28473699e-05, 7.95784911e-05],
       [1.20418969e+01, 1.76053615e+00, 1.11893259e+00, ...,
        3.03290722e-05, 2.48215042e-04, 7.95784911e-05],
       ...,
       [1.04602922e+02, 4.17220268e+00, 1.87086265e+00, ...,
        1.77679890e-02, 1.30254122e-02, 1.67313778e-02],
       [1.69855515e+01, 5.24290251e+00, 1.51687522e+00, ...,
        3.78300707e-05, 1.37645917e-04, 7.95784911e-05],
       [1.62941934e+01, 2.16769727e+00, 1.94017500e-01, ...,
        1.33840398e-02, 1.37534684e-02, 1.91187325e-02]])

In [63]:
f, ax = plt.subplots(figsize=(4, 3))
ax.plot(ss[0])
ax.twinx().plot(_ss[0], c='r')
ax.plot(__ss[0], c='k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
ss / _ss

array([[4.86563769e+31, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
        5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
        5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
        5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
        5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
        5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
        5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
        5.00000000e-01, 5.00000000e-01, 1.00000000e+00]])

In [84]:
dfn = sp.spatial.distance.pdist(ss)
dfn.shape

(11175,)

In [112]:
np.median(dfn)

7.053511365083138

In [113]:
100*0.3

30.0

In [10]:
window_length = 3
state_length = 0.1

In [11]:
# # # compute the power spectrum in each state_window with tukey tapering
# # _spec = ssn.spectrogram(
# #     unit_fr, fs=sampling_rate, window=('tukey', 1),
# #     nperseg=int(state_length*sampling_rate), noverlap=0,
# #     detrend=False, mode='psd', scaling='spectrum', axis=0
# # )[-1]
# # _spec_boxcar = ssn.spectrogram(
# #     unit_fr, fs=sampling_rate, window=('boxcar'),
# #     nperseg=int(state_length*sampling_rate), noverlap=0,
# #     detrend=False, mode='psd', scaling='spectrum', axis=0
# # )[-1]
# # # reshape the result to combine all neurons into a single vector x time
# # _spec_reshaped = np.reshape(_spec, (_spec.shape[0]*_spec.shape[1], -1))
# # _spec_boxcar_reshaped = np.reshape(_spec_boxcar, (_spec_boxcar.shape[0]*_spec_boxcar.shape[1], -1))
# # # drop some time at the end to make the shape compatible with an integral number of windows
# # _spec_reshaped = _spec_reshaped[:, :-int(_spec_reshaped.shape[1]%int(window_length / state_length))]
# # _spec_boxcar_reshaped = _spec_boxcar_reshaped[:, :-int(_spec_boxcar_reshaped.shape[1]%int(window_length / state_length))]
# # # reshape to divide the array into vectors x states x windows
# # _spec_final = np.reshape(_spec_reshaped, (_spec_reshaped.shape[0], -1, int(window_length / state_length)))
# # _spec_boxcar_final = np.reshape(_spec_boxcar_reshaped, (_spec_boxcar_reshaped.shape[0], -1, int(window_length / state_length)))

# # # compute pairwise distances between states for all windows
# # pdists = np.array([sp.spatial.distance.pdist(_spec_final[:, i, :].T) for i in range(_spec_final.shape[1])])
# # pdists_boxcar = np.array([sp.spatial.distance.pdist(_spec_boxcar_final[:, i, :].T) for i in range(_spec_final.shape[1])])
# # # compute median distance for all windows and normalize by number of units
# # diffn = np.median(pdists, axis=-1) / np.sqrt(unit_fr.shape[1])
# # diffn_boxcar = np.median(pdists_boxcar, axis=-1) / np.sqrt(unit_fr.shape[1])

# diffn = compute_differentiation(session_ids[0], param_sets[0:1], unit_set[0:1], diff_fn='tukD').iloc[:, 0]
# diffn_boxcar = compute_differentiation(session_ids[0], param_sets[0:1], unit_set[0:1], diff_fn='boxD').iloc[:, 0]

# # compute differentiation using the standard procedure that was used for all other analysis
# diffn_std = compute_differentiation(session_ids[0], param_sets[0:1], unit_set[0:1]).iloc[:, 0]
# # and using variance of mean firing rate as a metric
# diffn_var = compute_differentiation(session_ids[0], param_sets[0:1], unit_set[0:1], diff_fn='mfrD').iloc[:, 0]

# f, ax = plt.subplots(figsize=(12, 2), tight_layout=True)
# ax.plot(diffn.values/diffn.max(), label='tukey tapering')
# ax.plot(diffn_boxcar.values/diffn_boxcar.max(), label='no tapering')
# ax.plot(diffn_std.values/diffn_std.max(), label='standard')
# ax.plot(diffn_var.values/diffn_var.max(), label='variance of mfr')
# ax.set_xlabel('window #')
# ax.set_ylabel('differentiation')
# ax.legend(fontsize=9);

In [12]:
# f, (ax, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 3), tight_layout=True)
# ax.scatter(diffn_std, diffn, s=1, alpha=0.05, c='k')
# ax.set_xlabel('standard differentiation')
# ax.set_ylabel('differentiation with\ntukey tapering')

# ax2.scatter(diffn_std, diffn_boxcar, s=1, alpha=0.05, c='k')
# ax2.set_ylabel('differentiation\nwithout tapering')
# ax2.set_xlabel('standard differentiation')

# ax3.scatter(diffn_std, diffn_var, s=1, alpha=0.05, c='k')
# ax3.set_ylabel('variance of FR')
# ax3.set_xlabel('standard differentiation')

# f.savefig('fig_supp_comparison_tukey_boxcar_windows.pdf')

---

# Neuron numbers

In [13]:
allunits = {}
for session in tqdm(session_ids):
    allunits[session] = load_units(session)
allunits = pd.concat(allunits)

In [14]:
n_units_total = allunits[
    allunits.region.isin(region_sets['AllVis'])\
    &(allunits.snr>2.5)&(allunits.RS==True)
].groupby(level=0).apply(
    lambda df: df.groupby(['region', 'layer'], dropna=False).size()
).rename('n_units')
n_units_total

           region  layer
715093703  LGd     NaN       65
           LP      NaN      157
           VISam   L2/3       3
                   L4         4
                   L5        10
                           ... 
847657808  VISpm   L6        12
           VISrl   L2/3      12
                   L4         6
                   L5        22
                   L6        35
Name: n_units, Length: 1367, dtype: int64

In [15]:
n_units_layer = n_units_total.groupby(level=0).apply(
    lambda df: df.groupby('layer').sum()
).unstack()
display(n_units_layer.apply(
    lambda c: f'{c.mean():.0f} +/- {c.std():.0f}'
))

n_units_layer = n_units_layer.sum().dropna()
display(n_units_layer)
print(f'{n_units_layer.mean()} +/- {n_units_layer.std()}')

layer
L1       15 +/- 9
L2/3    52 +/- 25
L4      44 +/- 17
L5      73 +/- 26
L6      52 +/- 24
dtype: object

layer
L1       863
L2/3    3011
L4      2558
L5      4228
L6      3030
dtype: int64

2738.0 +/- 1217.3658036925467


In [16]:
n_units_area = n_units_total.groupby(level=0).apply(
    lambda df: df.groupby('region').sum()
).unstack()
display(n_units_area.apply(
    lambda c: f'{c.mean():.0f} +/- {c.std():.0f}'
))

n_units_area = n_units_area.sum().reindex(
    relevant_regions
).dropna().drop('TH')
display(n_units_area)
print(f'{n_units_area.mean()} +/- {n_units_area.std()}')

region
LGd      34 +/- 21
LP       54 +/- 41
TH         7 +/- 7
VISal    59 +/- 44
VISam    52 +/- 20
VISl     40 +/- 24
VISp     57 +/- 24
VISpm    46 +/- 23
VISrl    40 +/- 22
dtype: object

region
LGd      1087.0
LP       2197.0
VISp     3190.0
VISl     1685.0
VISrl    1991.0
VISal    2591.0
VISpm    1649.0
VISam    2584.0
dtype: float64

2121.75 +/- 663.6196737805258


---

# Differentiation

In [17]:
behavior_groups = {
    'running_speed' : [-40, 10, 1000],
}
behavior_keys = ['running_speed']

In [18]:
diff_func = 'specD' # 'specD' or 'mfrD' or 'boxD' or 'tukD'
min_units = 10

cutoffs  = {
    'specD' : 1e9,
    'boxD' : 1e7,
    'tukD' : 1e7,
    'mfrD' : 1e3,
}

# utility functions for properly renaming the columns of the differentiation dataframe
def get_unit_filters(units):
    filter_strings = units.split(' & ')
    filters = {}
    for filt in filter_strings:
        if ' = ' in filt:
            filters[filt.split(' = ')[0]] = filt.split(' = ')[1].strip('""')
        if '==' in filt:
            filters[filt.split(' == ')[0]] = filt.split(' == ')[1].strip('""')
        if '>' in filt:
            filters[filt.split(' > ')[0]] = float(filt.split(' > ')[1])
        if '@' in filt:
            key = filt.split(' ')[0]
            value = filt.split('get')[1].split('"')[1]
            filters[key] = value
    return filters

def rename_columns(c):
    props = get_unit_filters(c[2])
    return dict(
        window_length=c[0],
        state_length=c[1],
        layer=props.get('layer', '-'),
        area=props.get('region', '-'),
        n_units=int(props['n_units'])
    )

_stim_diff = []
_differentiation = {}
_differentiation_mean = {}
_n_units = {}
for session in tqdm(session_ids, desc='session'): # process all sessions
#     print(session)
    diff_fn = f'{data_directory}/{diff_func}{"_RS" if rs else "_FS"}_{session}.pkl'
    if path.exists(diff_fn): # force skip to append new unit_sets
        differentiation = pd.read_pickle(diff_fn)
        cparams = list(
            set(
                differentiation.columns.to_frame()
                .apply(
                    lambda r: (r.window_length, r.state_length), axis=1
                ).values
            )
        )
    else:
        differentiation = None
        cparams = []
    
    # what parameters are not yet computed?
    rparams = [
        p for p in param_sets if (
            p['WINDOW'], p['STATE_LENGTH']
        ) not in cparams
    ]
    if 0:#len(rparams) > 0:
#         print('Computing differentiation for ', rparams)
        rdifferentiation = compute_differentiation(
            session, rparams, unit_set, min_units, diff_fn=diff_func
        )
        if rdifferentiation is None:
            continue
        if differentiation is not None:
            differentiation = differentiation.join(rdifferentiation)
        else:
            differentiation = rdifferentiation
            
#         # for appending new unit_sets
#         try:
#             diffn = pd.read_pickle(diff_fn)
#             differentiation = diffn.join(differentiation).sort_index(axis=1)
#         except:
#             print(f'error for {session}')
#         # done appending new unit_sets
        display(differentiation)
        differentiation.to_pickle(diff_fn)
#         display(differentiation)

    if differentiation is None:
        continue
    differentiation.columns = pd.MultiIndex.from_frame(
        pd.DataFrame(
            list(differentiation.columns.to_frame().apply(rename_columns, axis=1))
        )
    )
    _n_units[session] = differentiation.columns.to_frame(index=False)

    # set areas in the order of hierarchy
    idx = differentiation.columns.to_frame()
    idx['area'] = idx.area.astype(
        pd.CategoricalDtype(hierarchy.keys(), ordered=True)
    )
    differentiation.columns = pd.MultiIndex.from_frame(idx)

    # add stimulus information to the index
    stim_table = load_stimulus_table(session)
    differentiation.index = pd.MultiIndex.from_frame(
        pd.MultiIndex.from_frame(stim_table)
        .drop_duplicates()
        .to_frame(index=False)
        .set_index('time')
        .reindex(
            differentiation.index, method='ffill'
        ).rename_axis('time').reset_index().bfill()
    )

    # functional connectivity dataset has a different name for drifting_gratings, so fix that
    idx = differentiation.index.to_frame()
    idx['stimulus_name'] = idx.stimulus_name.map(
        lambda x: x if x!='drifting_gratings_75_repeats' else 'drifting_gratings'
    )
    differentiation.index = pd.MultiIndex.from_frame(idx)

    # remove columns for stimuli that do not have differentiation computed
    differentiation = differentiation.drop(
        ['dot_motion', 'natural_scenes'], level='stimulus_name', errors='ignore'
    )

    # add stimulus differentiation column
    # load stimulus differentiation mean and SD
    stim_diff = pd.read_pickle(
        f'{data_directory}/spectral_differentiation_stimulus.pkl'
    ).rename_axis(['window_length', 'state_length', 'kind'], axis=1)
    stim_diff_std = stim_diff.xs('std_diff', axis=1, level='kind')
    stim_diff = stim_diff.xs('mean_diff', axis=1, level='kind')
    # from mean and SD, generate a random sampling of stimulus differentiation
    # and add it to the dataframe
    def get_stimulus_differentiation(df):
        stimulus = df.name
        size = len(df)
        stimdiff = {}
        for w in df.columns.levels[0]:
            for s in df.columns.levels[1]:
                try:
                    stimdiff[
                        (w, s, '-', 'stimulus', 192*130)
                    ] = np.random.normal(
                        stim_diff.loc[stimulus, (w, s)],
                        stim_diff_std.loc[stimulus, (w, s)],
                        size
                    )
                except:
                    pass
        return pd.DataFrame(stimdiff, index=df.index)
    stim_diff = differentiation.groupby('stimulus_name').apply(
        get_stimulus_differentiation
    ).fillna(0).rename_axis(
        ['window_length', 'state_length', 'layer', 'area', 'n_units'],
        axis=1
    )
    _stim_diff.append(stim_diff)
    differentiation = differentiation.join(stim_diff).sort_index(axis=1)

    # normalize differentiation by number of units
    if diff_func=='specD':
        differentiation = differentiation / \
        differentiation.columns.get_level_values(-1).map(np.sqrt)

    # order of stimuli by complexity (intuitive, for plotting)
    stimuli = [
        'spontaneous', 'flashes', 'drifting_gratings_contrast',
        'gabors', 'drifting_gratings', 'natural_movie_one_shuffled',
        'natural_movie_three', 'natural_movie_one', 'static_gratings'
    ]
    idx = differentiation.index.to_frame()
    idx['stimulus_name'] = idx.stimulus_name.astype(
        pd.CategoricalDtype(stimuli, ordered=True)
    )
    differentiation.index = pd.MultiIndex.from_frame(idx)

    # normalize for scaling with state length
    differentiation = differentiation /\
    differentiation.columns.get_level_values(
        'state_length'
    )**state_length_scaling_exponent
    
    try:
        running = load_running(session)
        running = running.set_index('times').reindex(
            differentiation.index.get_level_values(0),
            method='nearest'
        )
    except:
        print(f'Running data not found for {session}')
        running = pd.DataFrame(
            np.nan, columns=['running_speed'],
            index=differentiation.index.get_level_values('time')
        )
    # raw values of behavior parameters
    differentiation.index = pd.MultiIndex.from_frame(
        differentiation.index.to_frame().join(
            running.rename(lambda x: f'{x}_raw', axis=1)
        )
    )
    # digitized values of parameters
    differentiation.index = pd.MultiIndex.from_frame(
        differentiation.index.to_frame().join(
            running.apply(
                np.digitize, bins=behavior_groups['running_speed']
            )
        )
    )
    
    # keep only regions with more than 10 units
    differentiation = differentiation[differentiation.columns[
        differentiation.columns.get_level_values(-1)>=min_units
    ]].droplevel('n_units', axis=1)

    # For the first few seconds of certain recordings
    # (sessions 840012044, 839557629, 839068429), the value
    # of differentiation is 3-4 orders of magnitude larger
    # than at all other times / sessions. This is likely
    # due to some recording anomaly before the recording stabilizes?
    # Since the first few (up to 50) seconds are always spontaneous
    # activity, this affects only the spontaneous activity results.
    # Here we will replace such abnormal values of differentiation
    # with np.nan
    # You can comment out the code in this block and run the commented
    # code in the next to next cell to see which sessions / values
    # we are dropping
    differentiation[differentiation>cutoffs[diff_func]] = np.nan
    
    # compute mean differentiation by stimulus
    differentiation_mean = differentiation.groupby(
        level=behavior_keys+['stimulus_name']
    ).mean()
    
    _differentiation[session] = differentiation
    _differentiation_mean[session] = differentiation_mean

In [19]:
# build a single dataframe with mean differentiation by area and stimulus for all sessions
differentiation_mean_all_sessions = pd.concat(_differentiation_mean, names=['session'], sort=False)#.stack('session')
differentiation = pd.concat(_differentiation, names=['session']+list(list(_differentiation.values())[0].index.to_frame().columns))

# build aggregate dataframes with mean and SD of diff across sessions in each area/stimulus
differentiation_mean_all_behavior = differentiation_mean_all_sessions.groupby(behavior_keys+['stimulus_name']).mean()
differentiation_std_all_behavior = differentiation_mean_all_sessions.groupby(behavior_keys+['stimulus_name']).std()
differentiation_mean = differentiation_mean_all_behavior.loc[1]
differentiation_std = differentiation_std_all_behavior.loc[1]
differentiation_mean

window_length                         3                                \
state_length                      0.005                                 
layer                                  -                                
area                              AllVis          HVAs            LGd   
stimulus_name                                                           
spontaneous                 2.856452e+06  3.368470e+06  738281.102968   
flashes                     2.696889e+06  3.056754e+06  737316.728377   
drifting_gratings_contrast  3.197571e+06  3.729246e+06  781310.411822   
gabors                      2.650071e+06  3.115421e+06  642618.019620   
drifting_gratings           3.248925e+06  3.886014e+06  844950.051885   
natural_movie_one_shuffled  3.645685e+06  4.167867e+06  835750.717204   
natural_movie_three         3.302846e+06  3.823976e+06  954388.626582   
natural_movie_one           3.326980e+06  3.966425e+06  926345.105030   
static_gratings             3.532348e+06  4.360255e+06  935042.817898   

window_length                                                         \
state_length                                                           
layer                                                                  
area                                  LP           THx         VISal   
stimulus_name                                                          
spontaneous                 1.194367e+06  1.056148e+06  3.398182e+06   
flashes                     1.084070e+06  9.998742e+05  3.036117e+06   
drifting_gratings_contrast  1.154054e+06  1.085040e+06  3.630074e+06   
gabors                      1.017938e+06  9.330995e+05  3.197494e+06   
drifting_gratings           1.299962e+06  1.177154e+06  3.771072e+06   
natural_movie_one_shuffled  1.287839e+06  1.216583e+06  4.611200e+06   
natural_movie_three         1.577157e+06  1.341674e+06  3.864530e+06   
natural_movie_one           1.590552e+06  1.331934e+06  3.924420e+06   
static_gratings             1.638954e+06  1.343035e+06  4.118766e+06   

window_length                                                         \
state_length                                                           
layer                                                                  
area                               VISam          VISl          VISp   
stimulus_name                                                          
spontaneous                 2.832840e+06  2.994095e+06  3.224186e+06   
flashes                     2.293453e+06  2.583043e+06  2.610259e+06   
drifting_gratings_contrast  3.101996e+06  3.167388e+06  3.323541e+06   
gabors                      2.327847e+06  2.626154e+06  2.550028e+06   
drifting_gratings           3.293573e+06  3.274260e+06  3.502976e+06   
natural_movie_one_shuffled  3.348910e+06  3.752447e+06  4.645515e+06   
natural_movie_three         3.217995e+06  3.483701e+06  4.203650e+06   
natural_movie_one           3.388280e+06  3.607492e+06  4.263154e+06   
static_gratings             3.623780e+06  3.880126e+06  4.353225e+06   

window_length                             ...             30                 \
state_length                              ...         10.000                  
layer                                     ...             L5                  
area                               VISpm  ...          VISrl         VisCtx   
stimulus_name                             ...                                 
spontaneous                 2.485335e+06  ...  145033.522245  137968.416839   
flashes                     2.053961e+06  ...   91127.682079  101584.994955   
drifting_gratings_contrast  2.722870e+06  ...  148950.096600  157598.539281   
gabors                      1.899004e+06  ...  128094.184560  137951.091213   
drifting_gratings           2.852177e+06  ...  187621.160647  176815.236270   
natural_movie_one_shuffled  3.091636e+06  ...  125403.675295  173121.756511   
natural_movie_three         2.765266e+06  ...  248539.015335  244603.911864   
natural_mov

In [20]:
# up to first 50 seconds in the following three sessions, differentiation
# values are abnormally large. These are replaced by np.nan's to remove this artifact
display(differentiation[differentiation>cutoffs[diff_func]].dropna(how='all'))

x = differentiation[(3, 0.1, '-', 'AllVis')]#.xs('spontaneous', level='stimulus_name')

f, ax = plt.subplots(figsize=(4, 3), tight_layout=True)
x.hist(ax=ax, bins=50)
# ax.set_yscale('log')

Empty DataFrame
Columns: [(3, 0.005, -, AllVis), (3, 0.005, -, HVAs), (3, 0.005, -, LGd), (3, 0.005, -, LP), (3, 0.005, -, THx), (3, 0.005, -, VISal), (3, 0.005, -, VISam), (3, 0.005, -, VISl), (3, 0.005, -, VISp), (3, 0.005, -, VISpm), (3, 0.005, -, VISrl), (3, 0.005, -, VisCtx), (3, 0.005, -, hipp), (3, 0.005, -, stimulus), (3, 0.005, L2/3, HVAs), (3, 0.005, L2/3, VISal), (3, 0.005, L2/3, VISam), (3, 0.005, L2/3, VISl), (3, 0.005, L2/3, VISp), (3, 0.005, L2/3, VISpm), (3, 0.005, L2/3, VISrl), (3, 0.005, L2/3, VisCtx), (3, 0.005, L4, HVAs), (3, 0.005, L4, VISal), (3, 0.005, L4, VISam), (3, 0.005, L4, VISl), (3, 0.005, L4, VISp), (3, 0.005, L4, VISpm), (3, 0.005, L4, VISrl), (3, 0.005, L4, VisCtx), (3, 0.005, L5, HVAs), (3, 0.005, L5, VISal), (3, 0.005, L5, VISam), (3, 0.005, L5, VISl), (3, 0.005, L5, VISp), (3, 0.005, L5, VISpm), (3, 0.005, L5, VISrl), (3, 0.005, L5, VisCtx), (3, 0.005, L6, HVAs), (3, 0.005, L6, VISal), (3, 0.005, L6, VISam), (3, 0.005, L6, VISl), (3, 0.005, L6, VISp), (3, 0.005, L6, VISpm), (3, 0.005, L6, VISrl), (3, 0.005, L6, VisCtx), (3, 0.01, -, AllVis), (3, 0.01, -, HVAs), (3, 0.01, -, LGd), (3, 0.01, -, LP), (3, 0.01, -, THx), (3, 0.01, -, VISal), (3, 0.01, -, VISam), (3, 0.01, -, VISl), (3, 0.01, -, VISp), (3, 0.01, -, VISpm), (3, 0.01, -, VISrl), (3, 0.01, -, VisCtx), (3, 0.01, -, hipp), (3, 0.01, -, stimulus), (3, 0.01, L2/3, HVAs), (3, 0.01, L2/3, VISal), (3, 0.01, L2/3, VISam), (3, 0.01, L2/3, VISl), (3, 0.01, L2/3, VISp), (3, 0.01, L2/3, VISpm), (3, 0.01, L2/3, VISrl), (3, 0.01, L2/3, VisCtx), (3, 0.01, L4, HVAs), (3, 0.01, L4, VISal), (3, 0.01, L4, VISam), (3, 0.01, L4, VISl), (3, 0.01, L4, VISp), (3, 0.01, L4, VISpm), (3, 0.01, L4, VISrl), (3, 0.01, L4, VisCtx), (3, 0.01, L5, HVAs), (3, 0.01, L5, VISal), (3, 0.01, L5, VISam), (3, 0.01, L5, VISl), (3, 0.01, L5, VISp), (3, 0.01, L5, VISpm), (3, 0.01, L5, VISrl), (3, 0.01, L5, VisCtx), (3, 0.01, L6, HVAs), (3, 0.01, L6, VISal), (3, 0.01, L6, VISam), (3, 0.01, L6, VISl), (3, 0.01, L6, VISp), (3, 0.01, L6, VISpm), (3, 0.01, L6, VISrl), (3, 0.01, L6, VisCtx), (3, 0.015, -, AllVis), (3, 0.015, -, HVAs), (3, 0.015, -, LGd), (3, 0.015, -, LP), (3, 0.015, -, THx), (3, 0.015, -, VISal), (3, 0.015, -, VISam), (3, 0.015, -, VISl), ...]
Index: []

[0 rows x 1383 columns]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [21]:
def plot_dependence(
    stimulus_name, window_length, state_length, area, layer='-',
    kind='line', sort=False, sort_key=None, ax=None, lncol=3,
    dfm=differentiation_mean, dfs=differentiation_std, **pltargs
):
    kwargs = {
        k:v for k, v in locals().items() if k in [
            'stimulus_name', 'window_length',
            'state_length', 'area', 'layer'
        ]
    }
    title = {
        k:v for k, v in kwargs.items() if v not in [None, 'x', 'l', 'm']
    }
    legend = [k for k, v in kwargs.items() if v=='l']
    xlabel = [k for k, v in kwargs.items() if v=='x']
    columns = [i for i, v in enumerate([x for x in [
        window_length, state_length, layer, area, stimulus_name
    ] if x!='m']) if v!='x']
    nonmeans = [i for i, v in enumerate([
        window_length, state_length, layer, area, stimulus_name
    ]) if v!='m']
    p = [v if v not in ['x', 'l', 'm'] else None for v in [
        stimulus_name, window_length, state_length, area, layer
    ]]
    
    df_mean = dfm.unstack().loc[(
        slice(p[1], p[1], p[1]), slice(p[2], p[2], p[2]),
        slice(p[4], p[4]), slice(p[3], p[3]), slice(p[0], p[0])
    )].groupby(level=nonmeans).mean().unstack(columns)
    df_std = dfs.unstack().loc[(
        slice(p[1], p[1], p[1]), slice(p[2], p[2], p[2]),
        slice(p[4], p[4]), slice(p[3], p[3]), slice(p[0], p[0])
    )].groupby(level=nonmeans).mean().unstack(columns)
    
    if sort:
        if sort_key is None:
            df_mean = df_mean.loc[df_mean.mean(1).sort_values().index]
            df_std = df_std.loc[df_mean.mean(1).sort_values().index]
        else:
            df_mean = df_mean.loc[
                sorted(df_mean.index, key=lambda x: sort_key.get(x, 0))
            ]
            df_std = df_std.loc[
                sorted(df_std.index, key=lambda x: sort_key.get(x, 0))
            ]
    
    if ax is None:
        f, ax = plt.subplots(1, 1, figsize=(5, 2.5), tight_layout=True)
    kwargs = dict(
        line=dict(fmt='-o', ms=3, lw=2, alpha=0.6),
        bar=dict()
    )
    for v in kwargs.values():
        v.update(pltargs)
    try:
        df_mean.plot(
            yerr=df_std, kind=kind, ax=ax,
            label=window_length, **kwargs[kind]
        )
    except Exception as e:
        print(e)
        return ax, df_mean
    
    if len(legend)>0:
        ax.legend(
            labels=df_mean.columns.get_level_values(legend[0]),
            fontsize=8, title=legend[0], title_fontsize=9, ncol=lncol
        )
    ax.set_ylabel('differentiation')
    ax.set_xlabel(xlabel[0])
    ax.set_title(str(title)[1:-1].replace("'", ''))
    return ax, df_mean

# Characterizing the metric

## Dependence on window and state length

In [22]:
stim_autocorr_times = pd.read_pickle(
    path.join(data_directory, 'stimulus_autocorrelation.pkl')
)

In [23]:
stimulus = 'spontaneous'
area = 'VISp'
f, ax = plt.subplots(figsize=(3.5, 2.6), tight_layout=True)
_, df_mean = plot_dependence(
    stimulus_name=stimulus, window_length='m', state_length='x',
    area=area, layer='-', ax=ax,
    dfm=differentiation_mean_all_behavior.loc[1],
    dfs=differentiation_std_all_behavior.loc[1]
)
ax.set_title(f'{stimulus}', y=0.95)
ax.legend().set_visible(False)
ax.set_xscale('log')
ax.set_yscale('log', nonpositive='mask')
ax.set_xlabel('state length (s)')
ax.set_xlim(xmin=1e-2)
# ax.set_ylabel('')
# ax.set_yticks([])
ax.minorticks_off()
ax.axvline(1, c=cm.Greys(0.8, 0.5))
# ax.axvline(1, c=cm.Greys(0.8, 0.5))
sns.despine(ax=ax)
if 'window_length' not in df_mean.columns.names:
    _df = df_mean
elif len(df_mean[30])>0:
    _df = df_mean[30]
else:
    _df = None
try:
    if _df is not None:
        # seems like a power-law dependence, so let's plot the powerlaw
        x, y = np.log(_df.droplevel(
            ['area', 'stimulus_name'], axis=1
        ).reset_index()).values.T[:, -3:]
        m, c = np.polyfit(x, y, 1)
        _x = df_mean.index[-3:]
        _y = np.exp(m*np.log(_x)+c)*1.5
        ax.plot(_x, _y, c='r', alpha=0.5)
        ax.annotate(f'slope = {m:.1f}', (_x[-1], _y[1]), fontsize=7, ha='right', color='r', alpha=0.9)
except:
    pass

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
area = 'VISp'
layer = '-'
__df = []
f, axes = plt.subplots(3, 3, figsize=(10, 6), constrained_layout=True, sharex=True, sharey=False)
axes = {v:axes.flatten()[i] for i, v in enumerate(stimuli)}
for stimulus in stimuli:
    if stimulus not in differentiation_mean.index:
        axes[stimulus].set_xlabel('state length (s)')
        continue
    _, df_mean = plot_dependence(
        stimulus_name=stimulus, window_length='l', state_length='x',
        area=area, layer=layer, ax=axes[stimulus],
        dfm=differentiation_mean_all_behavior.loc[1], dfs=differentiation_std_all_behavior.loc[1]
    )
    __df.append(df_mean.loc[:, 3])
#     try:
#         axes[stimulus].axvline(stim_autocorr_times.mean()[stimulus], c=cm.Greys(0.8, 0.5))
#     except:
#         pass
    axes[stimulus].set_title(stimulus, y=0.95)
    axes[stimulus].legend().set_visible(False)
    axes[stimulus].set_xscale('log')
    axes[stimulus].set_yscale('log', nonpositive='mask')
    axes[stimulus].set_xlabel('state length (s)')
    axes[stimulus].set_ylabel('')
#     axes[stimulus].set_yticklabels([])
    axes[stimulus].minorticks_off()
    
    try:
        if len(df_mean[30])>0:
            # seems like a power-law dependence, so let's plot the powerlaw
            x, y = np.log(df_mean[30].droplevel(
                ['area', 'stimulus_name'], axis=1
            ).reset_index()).values.T[:, -4:]
            m, c = np.polyfit(x, y, 1)
            _x = df_mean.index[-4:]
            _y = np.exp(m*np.log(_x)+c)
            axes[stimulus].plot(_x, _y)
    #         axes[stimulus].annotate(f'slope = {m:.1f}', (_x[-1], _y[0]), fontsize=7, ha='right')
    except:
        pass
    
    sns.despine(ax=axes[stimulus])
axes['spontaneous'].legend(
    labels=['slope', '3s', '9s', '30s'],# title='window length',
    loc=(0, 0), fontsize=8, title_fontsize=9, frameon=False
)
# axes['spontaneous'].set_ylim(50, 2e4)
f.text(0.01, 0.5, 'differentiation', rotation=90, va='center')
f.suptitle(f'All Sessions (region: {area}; {layer if layer!="-" else "all layers"})', fontsize=11);
f.subplots_adjust(left=0.1);
f.savefig(f'fig_supp_visp_diff_stateln{"" if rs else "_FS"}{"" if diff_func=="specD" else "_"+diff_func}.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/numpy/core/_asarray.py:138: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order, subok=True)
/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/numpy/core/_asarray.py:138: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order, subok=True)
/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/numpy/core/_asarray.py:138: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order, subok=True)
/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/numpy/core/_asarray.py:138: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order, subok=True)
/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/numpy/core/_asarray.py:138: UserWarning: Warning: con

## Differentiation vs area

In [25]:
f, ax = plt.subplots(figsize=(7.5, 3), tight_layout=True)
ax, df_mean = plot_dependence(
    stimulus_name='l', window_length=3, state_length=0.1, ax=ax,
    area='x', kind='line', sort=True, sort_key=hierarchy, lw=2,
    color=differentiation_mean.index.astype(str).map(stim_colors),
    dfm=differentiation_mean_all_behavior.loc[1],
    dfs=differentiation_std_all_behavior.loc[1]
)
ax.set_xticks(range(len(differentiation_mean.columns.levels[-1])))
ax.set_xticklabels([
    x.replace('_', '\n') for x in sorted(
        differentiation_mean.columns.levels[-1],
        key=lambda x: hierarchy.get(x)
    )
], rotation=0, fontsize=8);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
f, ax = plt.subplots(figsize=(7.5, 3), tight_layout=True)
ax, df_mean = plot_dependence(
    stimulus_name='l', window_length=3, state_length=0.005, ax=ax,
    area='x', kind='line', sort=True, sort_key=hierarchy, lw=2,
    color=differentiation_mean.index.astype(str).map(stim_colors),
    dfm=differentiation_mean_all_behavior.loc[1],
    dfs=differentiation_std_all_behavior.loc[1]
)
ax.set_xticks(range(len(differentiation_mean.columns.levels[-1])))
ax.set_xticklabels([
    x.replace('_', '\n') for x in sorted(
        differentiation_mean.columns.levels[-1],
        key=lambda x: hierarchy.get(x)
    )
], rotation=0, fontsize=8);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
f, axes = plt.subplots(
    2, 1, figsize=(4.75, 2.7), sharex=True,
    constrained_layout=True
)#, gridspec_kw=dict(hspace=0.7))
for ax, state_length in zip(axes, [3, 0.005]):
    ax, _df = plot_dependence(
        stimulus_name='x', window_length='l', kind='bar', ax=ax,
        state_length=state_length, area='stimulus', layer='-'
    )
    if ax==axes[-1]:
        ax.legend(
            labels=[3, 9, 30], title='window length',
            fontsize=6, title_fontsize=6#, frameon=False
        )
    else:
        try:
            ax.get_legend().remove()
        except:
            pass
    ax.set_title(f'{state_length} s state length', fontsize=8)
    ax.set_ylabel('stimulus\ndifferentiation', fontsize=7)
    ax.tick_params(axis='both', labelsize=7, pad=0)
ax.set_xticklabels(
    ['spont' if x.get_text()=='spontaneous' else x.get_text().replace('_', '\n') for x in ax.get_xticklabels()],
    rotation=0, fontsize=7
)
ax.set_xlabel('');
f.savefig(f'fig_timescales_ordering{"" if diff_func=="specD" else "_"+diff_func}.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Draft figures: Dependence on hierarchy and stimulus complexity

In [28]:
if diff_func=='specD':
    scale_factor = 1e6
elif diff_func=='mfrD':
    scale_factor = 2
elif diff_func=='boxD':
    scale_factor = 200
elif diff_func=='tukD':
    scale_factor = 400
_dfn = differentiation[3][0.3]
_dfn.loc[:, (slice(None), 'stimulus')] = _dfn.loc[:, (slice(None), 'stimulus')]*scale_factor
_dfn = _dfn / scale_factor
_dfn = _dfn.loc[_dfn.drop('stimulus', level='area', axis=1).dropna(how='all').index]
_dfn

layer                                                                                      -  \
area                                                                                  AllVis   
session   time    stimulus_name              block running_speed_raw running_speed             
763673393 1.5     spontaneous                -1.0  0.391072          1              0.000000   
          4.5     spontaneous                -1.0  0.391072          1              0.294738   
          7.5     spontaneous                -1.0  0.391072          1              0.258215   
          10.5    spontaneous                -1.0  0.391072          1              0.707573   
          13.5    spontaneous                -1.0  0.391072          1              0.707220   
...                                                                                      ...   
799864342 10666.5 drifting_gratings_contrast  15.0 3.116136          1              0.699259   
          10669.5 drifting_gratings_contrast  15.0 3.116136          1              0.742872   
          10672.5 drifting_gratings_contrast  15.0 3.116136          1              0.464005   
          10675.5 drifting_gratings_contrast  15.0 3.116136          1              0.490622   
          10678.5 drifting_gratings_contrast  15.0 3.116136          1              0.550698   

layer                                                                                         \
area                                                                                    HVAs   
session   time    stimulus_name              block running_speed_raw running_speed             
763673393 1.5     spontaneous                -1.0  0.391072          1              0.000000   
          4.5     spontaneous                -1.0  0.391072          1              0.657010   
          7.5     spontaneous                -1.0  0.391072          1              0.648829   
          10.5    spontaneous                -1.0  0.391072          1              1.963480   
          13.5    spontaneous                -1.0  0.391072          1              2.651174   
...                                                                                      ...   
799864342 10666.5 drifting_gratings_contrast  15.0 3.116136          1              0.843424   
          10669.5 drifting_gratings_contrast  15.0 3.116136          1              0.861190   
          10672.5 drifting_gratings_contrast  15.0 3.116136          1              0.473270   
          10675.5 drifting_gratings_contrast  15.0 3.116136          1              0.459448   
          10678.5 drifting_gratings_contrast  15.0 3.116136          1              0.635796   

layer                                                                                         \
area                                                                                     LGd   
session   time    stimulus_name              block running_speed_raw running_speed             
763673393 1.5     spontaneous                -1.0  0.391072          1              0.000000   
          4.5     spontaneous                -1.0  0.391072          1              0.071024   
          7.5     spontaneous                -1.0  0.391072          1              0.074769   
          10.5    spontaneous                -1.0  0.391072          1              0.151809   
          13.5    spontaneous                -1.0  0.391072          1              0.109584   
...                                                                                      ...   
799864342 10666.5 drifting_gratings_contrast  15.0 3.116136          1              0.290255   
          10669.5 drifting_gratings_contrast  15.0 3.116136          1              0.304642   
          10672.5 drifting_gratings_contrast  15.0 3.116136          1              0.184882   
          10675.5 drifting_gratings_contrast  15.0 3.116136          1              0.210676   
          10678.5 drifting_gratings_contrast  15.0 3.116136   

In [29]:
# all differences are significant so we do not show
# significance levels in the plots; but it can be
# included by setting this to True
show_significance = True
if show_significance:
    import pingouin as pg
    
f = plt.figure(figsize=(10, 2.5), constrained_layout=True)
gs = plt.GridSpec(
    1, 5, figure=f,
    wspace=0.3, hspace=0.8
)

#----------------------------------------------------------------------------
sois = [
    'spontaneous', 'natural_movie_one_shuffled',
    'static_gratings', 'natural_movie_one'
]
rois = ['stimulus', 'THx', 'VISp', 'VisCtx', 'hipp']
# rois = ['stimulus', 'VISp', 'HVAs']
# rois = ['stimulus', 'THx', 'VISp', 'HVAs', 'hipp']
# rois = _dfn['-'].columns

palette = {s:stim_colors[s] for s in sois}

axes = [
    f.add_subplot(gs[0, i]) for i in range(5)
]
for ax in axes[2:]:
    ax.get_shared_y_axes().join(ax, axes[1])
    ax.tick_params(axis='y', labelleft=False)
for ax, region in zip(axes, rois):
    _df = _dfn['-'][
        _dfn.index
        .get_level_values('stimulus_name')
        .isin(sois)
    ].reset_index('stimulus_name')
    _df['stimulus_name'] = _df.stimulus_name.cat.remove_unused_categories()
    _df['stimulus_category'] = _df.stimulus_name.map(stimulus_categories)
    
    if show_significance:
        pval = pg.pairwise_gameshowell(
            _df[~_df[region].isna()], dv=region, between='stimulus_name'
        ).set_index(['A', 'B']).pval
        pval = pval.append(pval.swaplevel().rename_axis(['A', 'B']))
    
    sns.boxplot(
        x='stimulus_name', y=region, data=_df, ax=ax,
        order=sois, fliersize=0.4, linewidth=0.5,
        whis=2, palette=palette, showfliers=False
    )
    
    if show_significance:
        y0, y1 = ax.get_ylim()
        for i, a in enumerate(sois):
            for j, b in enumerate(sois[i+1:]):
                if pval.loc[(a, b)]<=0.001:
                    y = y0+(y1-y0)*(1+j*0.03+0.01*np.random.rand())
                    ax.plot([i+0.1, i+j+0.9], [y]*2, c='r', lw=0.5)
                    ax.plot([i+0.1]*2, [y*0.99, y], c='r', lw=0.5)
                    ax.plot([i+j+0.9]*2, [y*0.99, y], c='r', lw=0.5)
    
    ax.set_xticklabels([
        s.get_text()
        .replace('_', '\n') for s in ax.get_xticklabels()
    ])
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.set_title(region, fontsize=8)
    sns.despine(ax=ax)
    ax.tick_params(labelsize=7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.12, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [30]:
f = plt.figure(figsize=(8, 4), constrained_layout=True)
gs = plt.GridSpec(
    2, 5, figure=f, height_ratios=[3, 5],
    wspace=0.3, hspace=0.8
)

#----------------------------------------------------------------------------
sois = [
    'spontaneous', 'natural_movie_one_shuffled',
    'static_gratings', 'natural_movie_one'
]
rois = ['stimulus', 'THx', 'VISp', 'VisCtx', 'hipp']
# rois = ['stimulus', 'VISp', 'HVAs']
# rois = ['stimulus', 'THx', 'VISp', 'HVAs', 'hipp']
# rois = _dfn['-'].columns

palette = {s:stim_colors[s] for s in sois}

axes = [
    f.add_subplot(gs[0, i]) for i in range(5)
]
for ax in axes[2:]:
    ax.get_shared_y_axes().join(ax, axes[1])
    ax.tick_params(axis='y', labelleft=False)
for ax, region in zip(axes, rois):
    _df = _dfn['-'][
        _dfn.index.get_level_values('stimulus_name')
        .isin(sois)
    ].reset_index('stimulus_name')
    _df['stimulus_name'] = _df.stimulus_name.cat.remove_unused_categories()
    _df['stimulus_category'] = _df.stimulus_name.map(stimulus_categories)
    sns.boxplot(
        x='stimulus_name', y=region, data=_df, ax=ax,
        order=sois, fliersize=0.4, linewidth=0.5,
        whis=2, palette=palette, showfliers=False
    )
    
    ax.set_xticklabels([
        'spont', 'shuffled\nmovie', 'static\ngratings', 'natural\nmovie'
    ])
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.set_title(region, fontsize=8)
    sns.despine(ax=ax)
    ax.tick_params(labelsize=6)
    ax.yaxis.get_offset_text().set_size(7)
    
    if region=='hipp' and not rs:
        ax.set_title('hipp (NS)', fontsize=8)
axes[0].set_ylabel('SD', fontsize=8)
axes[1].set_ylabel('ND ( x $10^6$ )', fontsize=8)

#---------------------------------------------------------------------------
_dfn[('-', 'stimulus_categories')] = _dfn.index.get_level_values(
    'stimulus_name'
).map(stimulus_categories)
ordr = [x for x in hierarchy if x in _dfn['-'].columns]
ordr.remove('AllVis')
_df = _dfn['-'].reset_index(drop=True).set_index(
    'stimulus_categories', append=True
).stack().reset_index(['stimulus_categories', 'area'])

flierprops = dict(
    markerfacecolor=cm.Greys(0.5, 0.5),
    markeredgecolor='none'
)

ax = f.add_subplot(gs[1, :], sharey=axes[1])
sns.boxplot(
    x='area', y=0, hue='stimulus_categories', data=_df, ax=ax,
    order=ordr, fliersize=0.2, linewidth=0.2, whis=1.5,
    palette=stim_cat_colors_bg, width=0.7, flierprops=flierprops
)
ax.set_ylabel('ND ( x $10^6$ )', fontsize=8)
ax.set_xlabel('areas along visual hierarchy', fontsize=8)
sns.despine(ax=ax)

# crops out some outliers but all other relationships become clearer to see
if rs:
    ax.set_ylim(-0.08, 3.2)
else:
    ax.set_ylim(-0.045, 1.4)
ax.get_legend().remove()
ax.tick_params(labelsize=7)
ax.yaxis.get_offset_text().set_size(7)

with sns.axes_style('whitegrid'):
    ax2 = f.add_axes([0.125, 0.37, 0.12, 0.15])
sns.barplot(
    x='stimulus_categories', y=0,
    data=_df[_df.area=='stimulus'],
    palette=stim_cat_colors_bg, ax=ax2
)
ax2.set_ylabel('SD', fontsize=6.6, labelpad=-1)
ax2.set_xlabel('')
ax2.set_ylim(ymin=-0.05*ax2.get_ylim()[1])
ax2.set_xticklabels(
    ax2.get_xticklabels(), rotation=30, ha='right',
    rotation_mode='anchor', fontsize=6
)
ax2.set_yticks([0, 0.2])
ax2.set_yticklabels([0, 0.2], fontsize=7)
ax2.tick_params(axis='both', which='major', pad=-0.5);

f.text(0, 0.95, 'A', fontsize=10)
f.text(0, 0.5, 'B', fontsize=10)
f.align_ylabels([axes[0], ax])

f.savefig(f'fig_hierarchy_full{"" if rs else "_FS"}{"" if diff_func=="specD" else "_"+diff_func}.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy
    See https://zhiyzuo.github.io/Pearson-Correlation-CI-in-Python for reference
    Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''

    r, p = sp.stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(len(x)-3)
    z = sp.stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    return dict(r=r, p=p, ci_low=lo, ci_high=hi)

In [32]:
# is differentiation constant across the hierarchy or does it
# increase or decrease in a statistically significant manner?

def _unfold(lrr):
    return pd.Series(
        lrr#, index=['slope', 'intercept', 'rvalue', 'pvalue', 'stderr']
    )
# _df_reshaped = _df.rename(
#     {0:'differentiation'}, axis=1
# ).set_index(
#     ['stimulus_categories', 'area'], append=True
# ).unstack(
#     [2, 0]
# ).droplevel(0, axis=1)[region_sets['VisCtx']]
_df_reshaped = _df.groupby(
    ['stimulus_categories', 'area']
).mean().unstack('area').swaplevel(axis=1)[region_sets['VisCtx']]

_df_reshaped.columns = _df_reshaped.columns.map(
    lambda x: region_sets['VisCtx'].index(x[0])
)

_r = _df_reshaped.iloc[1]
regression_data = _df_reshaped.apply(
    lambda r: _unfold(
#         r.dropna()
        pearsonr_ci(r.dropna().index, r.dropna().values)
#         sp.stats.linregress(r.dropna().index, r.dropna().values)
    ),
    axis=1
)
pd.set_option("display.precision", 2)
display(regression_data)
pd.set_option("display.precision", 8)

,r,p,ci_low,ci_high
stimulus_categories,,,,
complex,-0.67,0.15,-0.96,0.31
natural,-0.87,0.03,-0.99,-0.19
shuffled,-0.52,0.29,-0.94,0.50
simple,-0.37,0.48,-0.91,0.63
spontaneous,-0.71,0.12,-0.96,0.25


# Draft figures: $\Delta$ differentiation heat maps

In [33]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm as anova
from scipy.stats import linregress, chi2

In [34]:
# the real juice (full analysis pipeline)
def get_models_and_fits(full, reduced, data, group):
    '''
    specify methods? ['bfgs', 'lbfgs', 'cg', 'powell'])
    '''
    mod_full = smf.mixedlm(full, data=data, groups=data[group])
    mod_full_fit = mod_full.fit(reml=False)
    print(
        'Convergence gradient for full model: ',
        mod_full.score(mod_full_fit.params_object)[0]
    )
    mod_red = smf.mixedlm(reduced, data=data, groups=data[group])
    mod_red_fit = mod_red.fit(reml=False)
    print(
        'Convergence gradient for reduced model: ',
        mod_red.score(mod_red_fit.params_object)[0]
    )
    
    chisq = (mod_full_fit.llf - mod_red_fit.llf)*2
    dfd = mod_full_fit.df_modelwc - mod_red_fit.df_modelwc
    pval = 1 - chi2.cdf(chisq, dfd)
    
    summary = pd.DataFrame(index=[full, reduced])
    summary['AIC'] = [mod_full_fit.aic, mod_red_fit.aic]
    summary['BIC'] = [mod_full_fit.bic, mod_red_fit.bic]
    summary['Log likelihood'] = [mod_full_fit.llf, mod_red_fit.llf]
    summary['dof'] = [mod_full_fit.df_modelwc, mod_red_fit.df_modelwc]
    summary['Chi Sq'] = [chisq, '']
    summary['p-value'] = [pval, '']
    display(summary)
    
    return dict(
        full_model = mod_full,
        reduced_model = mod_red,
        full_fit = mod_full_fit,
        reduced_fit = mod_red_fit,
    )

# construct the contrast matrix automatically
def triangle_prod(l):
    ret = []
    for i, item in enumerate(l):
        for j in range(i+1, len(l)):
            ret.append((item, l[j]))
    return ret

def get_contrast_matrix(
    effect_1, effect_2, data, fitted_model, effect_0=None
):
    # create contrast conditions to test
    if effect_0 is not None:
        contrast_conditions = [
            f'{e0} & {e1}' for e0, e1 in itertools.product(
                sorted(set(data[effect_0])), sorted(set(data[effect_1]))
            )
        ]
    else:
        contrast_conditions = sorted(set(data[effect_1]))
    contrast_between = sorted(set(data[effect_2]))

    contrasts = [
        f'{condition}: {type_1} - {type_2}'\
        for condition, (type_1, type_2) in itertools.product(
            contrast_conditions, triangle_prod(contrast_between)
        )
    ]
    
    terms = fitted_model.params.index.values[:-1]

    # define the contrast matrix
    contrast_matrix = pd.DataFrame(
        np.zeros([len(contrasts), len(terms)], dtype=int), 
        columns=terms,
        index=contrasts
    )

    # fill in the contrast matrix
    for contrast in contrast_matrix.index:
        if effect_0 is not None:
            e0, e1 = contrast.split(':')[0].split(' & ')
        else:
            e1 = contrast.split(':')[0]
        e2_1, e2_2 = contrast.split(': ')[1].split(' - ')
        for c in contrast_matrix.columns:
            # e1 is not a factor in this column, so set value only based on e2
            if effect_1 not in c:
                # first order terms with e0
                if (effect_0 is not None) and effect_0 in c:
                    if e0 in c and e2_1 in c:
                        contrast_matrix.loc[contrast, c] = 1
                    elif e0 in c and e2_2 in c:
                        contrast_matrix.loc[contrast, c] = -1
                # effect_0 and effect_1 not in c, so zeroth order terms
                else:
                    if e2_1 in c:
                        contrast_matrix.loc[contrast, c] = 1
                    elif e2_2 in c:
                        contrast_matrix.loc[contrast, c] = -1
            # only set value if both e1 and e2 match\
            else:
                # second order terms
                if (effect_0 is not None) and effect_0 in c:
                    if e0 in c and e1 in c and e2_1 in c:
                        contrast_matrix.loc[contrast, c] = 1
                    elif e0 in c and e1 in c and e2_2 in c:
                        contrast_matrix.loc[contrast, c] = -1
                # first order terms with e1
                else:
                    if e1 in c and e2_1 in c:
                        contrast_matrix.loc[contrast, c] = 1
                    elif e1 in c and e2_2 in c:
                        contrast_matrix.loc[contrast, c] = -1
    
    return contrast_matrix

def get_comparison_table(fitted_model, contrast_matrix):
    res = fitted_model.t_test(contrast_matrix).summary_frame()
    res.index = contrast_matrix.index
    return res

def get_disp_str(x):
    d = '{0:.2f}'.format(x['coef'])
    if x['P>|z|'] < 0.0001:
        d += '***'
    elif x['P>|z|'] < 0.001:
        d += '**'
    elif x['P>|z|'] < 0.01:
        d += '*'
    return d

In [35]:
def plot_heatmap(res, data, normalize_diff=True, pl=True):
    
    res['area'] = res.index.map(
        lambda x: x.split(':')[0].split(' & ')[1]
    )
    res['layer'] = res.index.map(
        lambda x: x.split(':')[0].split(' & ')[0]
    )
    res['stimulus_1'] = res.index.map(
        lambda x: x.split(': ')[1].split(' - ')[0]
    )
    res['stimulus_2'] = res.index.map(
        lambda x: x.split(': ')[1].split(' - ')[1]
    )

    res['display'] = res.apply(get_disp_str, axis=1)
    res.index = pd.MultiIndex.from_arrays([
        res.area, res.layer,
        res.stimulus_1, res.stimulus_2
    ])
    
    # plot the matrices
    df = res.display.apply(
        lambda x: float(x.strip('*'))
    ).unstack().T.stack()
    df = df.stack().swaplevel(0, 1).sort_index(level=-1).swaplevel(0, 1)
    if normalize_diff:
        df = (df.unstack() / data).stack()
    vmax = 1.5
    if df.drop(
        'stimulus', axis=1, errors='ignore'
    ).abs().values.max() < 1.5:
        vmax = 0.5
    if 'Stim' in df.columns:
        col_order = ['Stim']+list(
            df.mean().sort_values().index.drop('Stim')
        )
        df['Stim'] = -vmax + (df.Stim - df.Stim.min())\
        /(df.Stim.max() - df.Stim.min())*2*vmax
    else:
        col_order = df.columns#df.mean().sort_values().index
    df = df[col_order]
    sig = res.display.apply(lambda x: x.count('*'))\
    .unstack().T.stack().stack().swaplevel(0, 1)\
    .sort_index(level=-1).swaplevel(0, 1)

    dfs = diffn[diffn.area=='stimulus'].groupby(grouper)\
    .mean().differentiation.rename('stimulus').to_frame()
    dfs = -pd.DataFrame(
        dfs['stimulus'].values - dfs['stimulus'].values[:, None],
        index=dfs.index, columns=dfs.index
    ).stack().reindex(
        [tuple(x.split(' - ')) for x in diff_idx]
    ).dropna()
    
    if pl:
        f, ax = plt.subplots(
            1, len(df.index.levels[-1])+1,
            figsize=(8+4*(len(data.index.levels[1])>1), 4),
            constrained_layout=True, sharey=True,
            gridspec_kw=dict(
                width_ratios=[0.1]+[1]*len(df.index.levels[-1])
            )
        )
        axes = {'stimulus' : ax[0]}
        axes.update({
            df.index.levels[-1][i] : ax[i+1] for i in range(
                len(df.index.levels[-1])
            )
        })

        def _plot(_df):
            name = _df.name
            _df = _df.droplevel(-1)[[
                x for x in area_order if x in _df.columns
            ]]
            _sig = sig.xs(name, level='layer')
            _df = -pd.concat([_df, -_df.swaplevel()]).reindex(
                [tuple(x.split(' - ')) for x in diff_idx]
            ).swaplevel()#.dropna()
            _sig = pd.concat(
                [_sig, _sig.swaplevel()]
            ).reindex(_df.index).fillna(0)
            im = axes[name].imshow(
                _df, aspect='auto', cmap=cm.RdBu_r,
                vmin=-vmax, vmax=vmax
            )
            for c in range(len(_df.columns)):
                for r in range(len(_df.index)):
                    axes[name].text(
                        c, r, '*'*int(_sig.iloc[r, c]),
                        color='k', fontsize=10,
                        ha='center', va='center'
                    )
            axes[name].set_xticklabels(_df.columns)
            axes[name].set_xticks(range(len(_df.columns)))
            axes[name].set_title(name)
            axes[name].set_yticks(range(len(_df.index)))
            axes[name].set_yticklabels(
                _df.index.map(lambda x: f'{x[1]} - {x[0]}')
            )
            return im, _df
        _im = df.groupby(level=-1).apply(_plot)
        
        plt.colorbar(
            _im.iloc[-1][0], ax=list(axes.values())[-1],
            label='spectral differentiation (normalized)',
            aspect=50
        )

        norm = mpl.colors.TwoSlopeNorm(
            0, dfs.min(), dfs.max()
        )
        axes['stimulus'].imshow(
            dfs.reindex(
                _im.iloc[0][1].index.swaplevel()
            ).to_frame(),
            aspect='auto', cmap=cm.RdBu_r, norm=norm
        )
        axes['stimulus'].set_xticks([0])
        axes['stimulus'].set_xticklabels(['stimulus'])
        hs = []
        for ax in axes.values():
            s = 0
            for h in reversed(
                range(len(
                    stim_by_putative_meaning[grouper]
                ))
            ):
                ax.axhline(s-0.5, c='k')
                s += h
                hs.append(s)
        for y in hs:
            line = mpl.lines.Line2D(
                [-2, -1], [y-0.5, y-0.5], c='k', lw=1
            )
            line.set_clip_on(False)
            axes['stimulus'].add_line(line)

        f.suptitle(
            f'state length {state_length}s, window length {window_length}s, '+str(filters)[1:-1].replace("'", '')
        )
    else:
        f = None
    
    return res, df, f, dfs

def get_reorganized_res(
    results, data, normalize_diff=True, pl=True
):
    _agg = {
        k : plot_heatmap(
            results[k].copy(), v, pl=pl
        ) for k, v in data.items()
    }
    res = {k:v[0] for k, v in _agg.items()}
    df = {k:v[1] for k, v in _agg.items()}
    figs = {k:v[2] for k, v in _agg.items()}
    dfs = {k:v[3] for k, v in _agg.items()}
    
    _df_dict = {
        k:-pd.concat([
            v.unstack(level='layer'), -v.unstack(level='layer').swaplevel()
        ]).stack().swaplevel(0, -1).unstack(-2) for k, v in df.items()
    }
    _dfl_dict = {
        k:pd.concat([
            v['display'].unstack(level='layer'),
            v['display'].unstack(level='layer').swaplevel()
        ]).stack().swaplevel(0, -1).unstack([-1, -3]).applymap(
            lambda x: str(x).count('*')*'*'
        ) for k, v in res.items()
    }
    
    _df = pd.concat(_df_dict).droplevel(0).rename(index={'-':'all layers'})
    _dfl = pd.concat(_dfl_dict).droplevel(0).rename(index={'-':'all layers'})
    
    return res, df, figs, _df, _dfl, dfs

In [36]:
grouper = 'stimulus_category'
area_order = [
    'LGd', 'LP', 'THx', 'THx_VISp', 'VISp', 'VISl', 'VISrl', 'VISal',
    'VISpm', 'VISam', 'HVAs', 'VisCtx', 'AllVis', 'hipp'
]
diff_idx = [f'{s1} - {s2}' for s1, s2 in itertools.product(
    stim_by_putative_meaning[grouper][::-1], repeat=2
) if stim_by_putative_meaning[grouper].index(s1)>stim_by_putative_meaning[grouper].index(s2)]

In [37]:
window_length = 3
state_length = 0.3

diffn = differentiation[
    (window_length, state_length)
]

diffn = diffn.stack(
    ['layer', 'area']
).index.to_frame().join(
    diffn.stack(
        ['layer', 'area']
    ).rename('differentiation')
).reset_index(drop=True)

diffn['stimulus_name'] = diffn.stimulus_name.map(
    lambda x: 'natural_movies' if 'natural_movie' in x and 'shuffled' not in x else x
)
diffn['stimulus_category'] = diffn.stimulus_name.map(stimulus_categories)

diffn

/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2855: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


,session,time,stimulus_name,block,running_speed_raw,running_speed,layer,area,differentiation,stimulus_category
0,763673393,1.5,spontaneous,-1.0,0.39107248,1,-,AllVis,0.00000000e+00,spontaneous
1,763673393,1.5,spontaneous,-1.0,0.39107248,1,-,HVAs,0.00000000e+00,spontaneous
2,763673393,1.5,spontaneous,-1.0,0.39107248,1,-,LGd,0.00000000e+00,spontaneous
3,763673393,1.5,spontaneous,-1.0,0.39107248,1,-,THx,0.00000000e+00,spontaneous
4,763673393,1.5,spontaneous,-1.0,0.39107248,1,-,VISam,0.00000000e+00,spontaneous
...,...,...,...,...,...,...,...,...,...,...
4706323,799864342,10678.5,drifting_gratings_contrast,15.0,3.11613612,1,L5,VisCtx,4.51296624e+05,simple
4706324,799864342,10678.5,drifting_gratings_contrast,15.0,3.11613612,1,L6,HVAs,3.13178459e+06,simple
4706325,799864342,10678.5,drifting_gratings_contrast,15.0,3.11613612,1,L6,VISal,4.56787174e+06,simple
4706326,799864342,10678.5,drifting_gratings_contrast,15.0,3.11613612,1,L6,VISp,6.49744863e+05,simple


In [38]:
# specD
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 5.2), tight_layout=True, sharex=True)

_df = diffn[diffn.stimulus_name=='spontaneous'].differentiation
_df[(_df<_df.quantile(0.9999))&(_df>0)].apply(np.log).hist(ax=ax1, log=True, bins=100)
ax1.set_title('spontaneous')

_df = diffn[diffn.stimulus_name=='natural_movies'].differentiation
_df[(_df<_df.quantile(0.9999))&(_df>0)].apply(np.log).hist(ax=ax2, log=True, bins=100)
ax2.set_title('natural_movies')

f.suptitle(diff_func);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
# tukD
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 5.2), tight_layout=True, sharex=True)

_df = diffn[diffn.stimulus_name=='spontaneous'].differentiation
_df[(_df<_df.quantile(0.9999))&(_df>0)].apply(np.log).hist(ax=ax1, log=True, bins=100)
ax1.set_title('spontaneous')

_df = diffn[diffn.stimulus_name=='natural_movies'].differentiation
_df[(_df<_df.quantile(0.9999))&(_df>0)].apply(np.log).hist(ax=ax2, log=True, bins=100)
ax2.set_title('natural_movies')

f.suptitle(diff_func);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
# boxD
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 5.2), tight_layout=True, sharex=True)

_df = diffn[diffn.stimulus_name=='spontaneous'].differentiation
_df[(_df<_df.quantile(0.9999))&(_df>0)].apply(np.log).hist(ax=ax1, log=True, bins=100)
ax1.set_title('spontaneous')

_df = diffn[diffn.stimulus_name=='natural_movies'].differentiation
_df[(_df<_df.quantile(0.9999))&(_df>0)].apply(np.log).hist(ax=ax2, log=True, bins=100)
ax2.set_title('natural_movies')

f.suptitle(diff_func);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
# mfrD
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 5.2), tight_layout=True, sharex=True)

_df = diffn[diffn.stimulus_name=='spontaneous'].differentiation
_df[(_df<_df.quantile(0.9999))&(_df>0)].apply(np.log).hist(ax=ax1, log=True, bins=100)
ax1.set_title('spontaneous')

_df = diffn[diffn.stimulus_name=='natural_movies'].differentiation
_df[(_df<_df.quantile(0.9999))&(_df>0)].apply(np.log).hist(ax=ax2, log=True, bins=100)
ax2.set_title('natural_movies')

f.suptitle(diff_func);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
# build a subframe for MLE models analysis

# filters for subselecting data according to behavior
filters = dict()
# filters.update(dict(running_speed=1))

data = {
    'split_layers' : diffn[
        diffn.layer.isin(['L2/3', 'L4', 'L5', 'L6'])
    ].dropna(),
    'combined_layers' : diffn[
        diffn.layer.isin(['-'])
    ].dropna()
}

_data = {
    k:v.copy() for k, v in data.items()
}

for f, v in filters.items():
    for k in data.keys():
        data[k] = data[k][data[k][f]==v]

data = {
    k:v.append(
        _data[k][_data[k].area=='stimulus']
    ) for k, v in data.items()
}

{display(v.head()) for v in data.values()};

,session,time,stimulus_name,block,running_speed_raw,running_speed,layer,area,differentiation,stimulus_category
11,763673393,1.5,spontaneous,-1.0,0.39107248,1,L2/3,HVAs,0.0,spontaneous
12,763673393,1.5,spontaneous,-1.0,0.39107248,1,L2/3,VISp,0.0,spontaneous
13,763673393,1.5,spontaneous,-1.0,0.39107248,1,L2/3,VISrl,0.0,spontaneous
14,763673393,1.5,spontaneous,-1.0,0.39107248,1,L2/3,VisCtx,0.0,spontaneous
15,763673393,1.5,spontaneous,-1.0,0.39107248,1,L4,HVAs,0.0,spontaneous


,session,time,stimulus_name,block,running_speed_raw,running_speed,layer,area,differentiation,stimulus_category
0,763673393,1.5,spontaneous,-1.0,0.39107248,1,-,AllVis,0.0,spontaneous
1,763673393,1.5,spontaneous,-1.0,0.39107248,1,-,HVAs,0.0,spontaneous
2,763673393,1.5,spontaneous,-1.0,0.39107248,1,-,LGd,0.0,spontaneous
3,763673393,1.5,spontaneous,-1.0,0.39107248,1,-,THx,0.0,spontaneous
4,763673393,1.5,spontaneous,-1.0,0.39107248,1,-,VISam,0.0,spontaneous


In [43]:
effect_0 = 'layer'
effect_1 = 'area'
effect_2 = grouper
response = 'differentiation'
random = 'session'

full = f'{response} ~ {effect_0}*{effect_1}*{effect_2}'
reduced = f'{response} ~ {effect_0}+{effect_1}+{effect_2}'

s = f'pub_results{"_RS" if rs else "_FS"}_{window_length}_{state_length}_{effect_0}_{effect_1}_{effect_2}_{response}_{random}'
for i, v in filters.items():
    s += f'_{i}_{v}'
s += f'{"" if diff_func=="specD" else "_"+diff_func}.pkl'
if 0:#path.exists(s):
    with open(s, 'rb') as f:
        results, data = pickle.load(f)
else:
    results = {
        k:get_models_and_fits(
            full, reduced, group=random,
            data=v
        ) for k, v in data.items()
    }
    results = {k:v['full_fit'] for k, v in results.items()}
    r1 = results
    results = {
        k:get_comparison_table(
            v, get_contrast_matrix(
                effect_1, effect_2, fitted_model=v,
                effect_0=effect_0, data=data[k]
            )
        ) for k, v in results.items()}
    r2 = results
    data = {
        k:v.groupby(
            ['area', 'layer']
        ).differentiation.mean() for k, v in data.items()
    }
    with open(s, 'wb') as f:
        pickle.dump((results, data), f)

# print('Summary of full model fitting:')
# display(results['full_fit'].summary())
# print('\n\nSummary of reduced model fitting:')
# display(results['reduced_fit'].summary())

Convergence gradient for full model:  -1.4322708513011688
Convergence gradient for reduced model:  -3.8402614047044517


,AIC,BIC,Log likelihood,dof,Chi Sq,p-value
differentiation ~ layer*area*stimulus_category,8.31566825e+07,8.31587494e+07,-4.15781793e+07,161,69110.531,0
differentiation ~ layer+area+stimulus_category,8.32255031e+07,8.32257199e+07,-4.16127345e+07,16,,


Convergence gradient for full model:  -9.549412535515078
Convergence gradient for reduced model:  -10.699818881193456


,AIC,BIC,Log likelihood,dof,Chi Sq,p-value
differentiation ~ layer*area*stimulus_category,5.20532511e+07,5.20541445e+07,-2.60265535e+07,71,37869.595,0
differentiation ~ layer+area+stimulus_category,5.20910167e+07,5.20912649e+07,-2.60454883e+07,19,,


In [44]:
res, df, figs, _df, _dfl, dfs = get_reorganized_res(results, data, pl=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/ipykernel_launcher.py:95: UserWarning: FixedFormatter should only be used together with FixedLocator


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/ipykernel_launcher.py:95: UserWarning: FixedFormatter should only be used together with FixedLocator


In [45]:
results['split_layers'].sort_index()

,coef,std err,z,P>|z|,Conf. Int. Low,Conf. Int. Upp.
L2/3 & HVAs: complex - natural,2.24299738e+05,22211.40724650,10.09840284,5.61491868e-24,1.80766180e+05,2.67833296e+05
L2/3 & HVAs: complex - shuffled,2.03470679e+05,38683.78759199,5.25984377,1.44177850e-07,1.27651849e+05,2.79289509e+05
L2/3 & HVAs: complex - simple,3.43988993e+05,18766.71789397,18.32973647,4.79221084e-75,3.07206902e+05,3.80771085e+05
L2/3 & HVAs: complex - spontaneous,3.82873095e+05,18340.49951548,20.87582700,8.88187425e-97,3.46926377e+05,4.18819814e+05
L2/3 & HVAs: natural - shuffled,-2.08290590e+04,41750.73411989,-0.49889085,6.17856284e-01,-1.02658994e+05,6.10008762e+04
...,...,...,...,...,...,...
L6 & VisCtx: natural - simple,1.85060483e+06,24003.58934398,77.09700417,0.00000000e+00,1.80355866e+06,1.89765100e+06
L6 & VisCtx: natural - spontaneous,2.18498284e+06,23682.73437667,92.26058125,0.00000000e+00,2.13856553e+06,2.23140015e+06
L6 & VisCtx: shuffled - simple,1.59862010e+06,39137.06711615,40.84670163,0.00000000e+00,1.52191286e+06,1.67532735e+06
L6 & VisCtx: shuffled - spontaneous,1.93299811e+06,38901.50508162,49.68954571,0.00000000e+00,1.85675257e+06,2.00924366e+06


In [46]:
# individual areas vs combined area differentiation
hvas = ['VISl', 'VISrl', 'VISal', 'VISpm', 'VISam']
f, ax = plt.subplots(figsize=(11, 3.5), tight_layout=True)
df['combined_layers'][hvas].plot(lw=0, marker='*', ax=ax)
df['combined_layers'][hvas].median(1).rename('median').plot(
    lw=1, marker='*', ax=ax, c='k', ls='none',
    yerr=df['combined_layers'][hvas].mad(1)
)
df['combined_layers'][hvas].mean(1).rename('mean').plot(
    lw=1, marker='*', ax=ax, c='k', alpha=0.6, ls='none',
    yerr=df['combined_layers'][hvas].std(1)
)
df['combined_layers']['HVAs'].plot(ax=ax, c=cm.Greys(0.5, 0.5))
ax.legend(fontsize=9, loc=(1.01, 0))
ax.set_xticks(range(len(df['combined_layers'])))
ax.set_xticklabels(df['combined_layers'].index.map(
    lambda x: f'{x[1]}\n-{x[0]}'
))
ax.set_ylabel('$\Delta$ differentiation')
ax.set_xlabel('area pairs')
ax.axhline(0, c=cm.Greys(0.1, 0.8));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
class MidpointNormalize(mpl.colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
    
areas = [
    'VISp', 'VISl', 'VISrl', 'VISal', 'VISpm',
    'VISam', 'VisCtx', 'HVAs', 'AllVis'
]

with sns.axes_style('darkgrid'):
    fig, axes = plt.subplots(
        5, len(areas), figsize=(8.5, 5),
        sharex=True, sharey=True, tight_layout=True,
        gridspec_kw={'hspace':0, 'wspace':0}
    )
    axes = axes.T
    AX = fig.add_axes([0.85, 0.15, 0.1, 0.17])
    cax = fig.add_axes([0.85, 0.4, 0.014, 0.5])
    for il, l in enumerate(_df.index.levels[0]):
        for ia, a in enumerate(areas):
            
            if a in ['HVAs', 'AllVis']:
                axes[ia, il].axis('off')
                continue
            axes[ia, il].set_xlabel('')
            axes[ia, il].set_ylabel('')
            if il==0:
                axes[ia, il].set_title(a, fontsize=8)
            else:
                axes[ia, il].axhline(0, c=cm.Greys(0.7, 0.8))
            if ia==0:
                axes[ia, il].set_ylabel(l, fontsize=8)
            else:
                axes[ia, il].axvline(0, c=cm.Greys(0.7, 0.8))
            axes[ia, il].tick_params(
                left=False, bottom=False, labelsize=6, pad=-1
            )
            _ddf = _df.loc[l, a].loc[
                stim_by_putative_meaning['stimulus_category'][0:]
            ][stim_by_putative_meaning['stimulus_category']]
            _ddfl = _dfl.loc[l, a].loc[
                stim_by_putative_meaning['stimulus_category'][0:]
            ][stim_by_putative_meaning['stimulus_category']]
            
            sns.heatmap(
                np.tril(_ddf, k=-1), ax=axes[ia, il], fmt='', cmap=cm.RdBu_r,
                norm=MidpointNormalize(midpoint=0, vmin=-0.2, vmax=1), annot=_ddfl,
                mask=np.triu(_ddf, k=0)!=0, annot_kws={'size':5}, linewidths=.5,
                cbar_ax=None if ia+il else cax, cbar=False if ia+il else True,
#                 cbar_kws=dict(
#                     label=
#                 )
            )
cax.tick_params(labelsize=8)
cax.set_ylabel(r'$\frac{\Delta ~differentiation}{differentiation}$', fontsize=11)
for l, ax in zip(_df.index.levels[0], axes[0, :]):
    ax.set_yticks(np.arange(len(_ddf.index.values))+0.5)
    ax.set_yticklabels(_ddf.index.values, rotation=0)
    ax.set_ylabel(l, fontsize=8)
for ax in axes[:, -1]:
    ax.set_xticks(np.arange(len(_ddf.index))+0.5)
    ax.set_xticklabels(
        _ddf.index.values, rotation=40,
        ha='right', rotation_mode='anchor'
    )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/seaborn/matrix.py:332: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  ax.figure.draw(ax.figure.canvas.get_renderer())


---

In [48]:
results, data, res, df, _df, _dfl, dfs = {}, {}, {}, {}, {}, {}, {}
for s in glob(
    f'pub_results{"_RS" if rs else "_FS"}_{window_length}_{state_length}_{effect_0}_{effect_1}_{effect_2}_{response}_{random}*'
):
    try:
        filt = s.rstrip('.pkl').split(f'{random}_')[1]
    except:
        filt = 'no_filter'
    with open(s, 'rb') as f:
        results[filt], data[filt] = pickle.load(f)
        res[filt], df[filt], _, _df[filt], _dfl[filt], dfs[filt] = get_reorganized_res(results[filt], data[filt], pl=False)
_dfl = pd.concat(_dfl, names=['filter', 'layer', 'stimulus_2'])
_df = pd.concat(_df, names=['filter', 'layer', 'stimulus_2'])

for k in dfs.keys():
    print(k)
    _x = dfs[k]['combined_layers']
    _x = pd.concat(
        [_x, -_x.swaplevel()]
    ).unstack().rename_axis('stimulus_2').rename_axis('stimulus_1', axis=1)
    for c in _df.loc[(k, 'all layers'), 'stimulus'].columns:
        _df.loc[(k, 'all layers'), ('stimulus', c)] = _x[c].values
    _df.loc[(k, 'all layers'), 'stimulus']

running_speed_1
no_filter
running_speed_2
tukD
mfrD
boxD


In [49]:
def plot_qtriangle(filt, area, layer, _df, _dfl, ax):
    _ddf = _df.loc[filt].loc[layer, area].loc[
        stim_by_putative_meaning['stimulus_category'][0:]
    ][stim_by_putative_meaning['stimulus_category']]
    _ddfl = _dfl.loc[filt].loc[l, a].loc[
        stim_by_putative_meaning['stimulus_category'][0:]
    ][stim_by_putative_meaning['stimulus_category']]
    sns.heatmap(
        np.tril(_ddf, k=-1), ax=ax, cmap=cm.RdBu_r, mask=np.triu(_ddf, k=0)!=0,
        cbar=False, annot=_ddfl, fmt='', annot_kws={"size":5, "color":"k"},
        linewidths=.5, norm=MidpointNormalize(midpoint=0, vmin=-0.1, vmax=1)
    )
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    ax.set_yticks(np.arange(len(_ddf.index.values))+0.5)
    ax.set_yticklabels(_ddf.index.values, rotation=0, fontsize=7)
    
    ax.set_xticks(np.arange(len(_ddf.index))+0.5)
    ax.set_xticklabels(
        _ddf.index.values, rotation=40,
        ha='right', rotation_mode='anchor'
    )
    
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=60, fontsize=7, ha='right')
#     ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=7)

In [50]:
s = ''
for k, v in filters.items():
    s += f'__{k}__{v}'
s

''

In [51]:
with sns.axes_style('darkgrid'):
    f, ax = plt.subplots(figsize=(1.5, 1.5), constrained_layout=True)
ax = AX
plot_qtriangle('no_filter', 'stimulus', 'all layers', _df, _dfl, ax)
ax.set_title('stimulus', fontsize=8)
ax.set_xticklabels(ax.get_xticklabels(), fontsize=6, rotation=30, rotation_mode='anchor')
ax.set_yticklabels(ax.get_yticklabels(), fontsize=6)#, rotation=30)
ax.tick_params(pad=-1);
fig.text(0, 0.5, s, rotation=90, va='center')
fig.savefig(f'fig_layers_areas{s}{"" if diff_func=="specD" else "_"+diff_func}.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/seaborn/matrix.py:332: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  ax.figure.draw(ax.figure.canvas.get_renderer())
/home/saurabh.gandhi/Projects/.env_difflib/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  # Remove the CWD from sys.path while we load stuff.


In [52]:
# filter_names = ['running_speed']
# filters = _df.index.levels[0]
# for filt in filter_names:
#     filts = ['no_filter']+list(filters[filters.str.contains(filt)])
#     for layer in ['L2/3']:#_df.index.levels[1]:
#         for area in areas:
#             with sns.axes_style('darkgrid'):
#                 f, axes = plt.subplots(1, len(filts), figsize=(3*len(filts), 2.8), constrained_layout=True, squeeze=False)
#             axes = axes[0]
#             for ax, fl in zip(axes, filts):
#                 plot_qtriangle(fl, area, layer, _df, _dfl, ax)
#                 ax.set_title(f'{layer} {area} {fl}', fontsize=8)

---